### Adapt the orcasound annotations file to our format

In [57]:
import pandas as pd
import numpy as np
import soundfile as sf

In [53]:
annotation_file = pd.read_csv('train_data/annotations.tsv', sep='\t')
example_file = pd.read_csv('../../../finding_willy/EDA/combined_annotations.csv')

#### remove calls with duration of 2.450 (not precise, comes from a dl model with low temporal resolution)

In [65]:
suspects = annotation_file[np.isclose(annotation_file['duration_s'],2.450)]
annotation_file = annotation_file[~(np.isclose(annotation_file['duration_s'],2.450))]

In [68]:
annotation_file.rename(columns={'wav_filename': 'filename', 
                                'start_time_s':'begin_time', 
                                'duration_s': 'call_length'}, inplace=True)

/home/moshe/.pyenv/versions/pywhale/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [70]:
annotation_file['end_time'] = annotation_file['begin_time'] + annotation_file['call_length']

<ipython-input-70-a4c9fa977cf9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_file['end_time'] = annotation_file['begin_time'] + annotation_file['call_length']


In [71]:
annotation_file['label'] = np.ones((len(annotation_file),), dtype=int)

<ipython-input-71-182fc818c42c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation_file['label'] = np.ones((len(annotation_file),), dtype=int)


#### reorder the columns

In [72]:
cols = annotation_file.columns.tolist()
orig_cols = example_file.columns.tolist()
remaining = list(set(cols) - set(orig_cols))
new_cols = orig_cols + remaining

In [73]:
annotation_file = annotation_file[new_cols]

#### remove .wav extension from the file names

In [74]:
name_series = [x.replace('.wav', '') for x in annotation_file['filename']]
annotation_file['filename'] = name_series

### label bg as bg, solve begin_time and call_len issues

In [75]:
annotation_file.loc[annotation_file['call_length'] == 0, 'label'] = 0

In [76]:
bg_files = list(annotation_file[annotation_file['label'] == 0]['filename'])

In [77]:
len(bg_files)

636

In [78]:
import soundfile as sf

In [80]:
wavs_path = 'train_data/wav'

In [81]:
for bg_file in bg_files:
    annotation_file.loc[annotation_file['filename'] == bg_file, 'end_time'] = sf.info(f'{wavs_path}/{bg_file}.wav').duration
    annotation_file.loc[annotation_file['filename'] == bg_file, 'call_length'] = sf.info(f'{wavs_path}/{bg_file}.wav').duration

In [90]:
files_3 = list(annotation_file['filename'])
srs = []
for file in files_3:
    srs.append(sf.info(f'{wavs_path}/{file}.wav').samplerate)
srs = set(srs)
print(srs)
assert len(srs)==1, 'Should contain only one sample rate across the dataset!'

{20000}


In [88]:
annotation_file.to_csv('orcasound_reformated.csv', index=False)